In [1]:
import numpy as np

from constants import FLOAT_DTYPE, INT_DTYPE, EPOCHS
from Network import CNN
from Layers import MaxPoolLayer, Conv2D, Dropout, Relu, Softmax
from dataset import get_2D_normalised

c:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
(train_data, train_labels), (test_data, test_labels) = get_2D_normalised()

In [4]:
# Convert labels to one-hot encoded vectors
num_classes = 10
train_labels_enc = np.eye(num_classes)[train_labels].reshape(-1, num_classes)
test_labels_enc = np.eye(num_classes)[test_labels].reshape(-1, num_classes)

In [5]:
# Create train and validation sets
val_split = 0.2
num_examples = train_data.shape[0]
val_size = int(val_split * num_examples)
train_size = num_examples - val_size

x_train = FLOAT_DTYPE(train_data[:train_size])
y_train = INT_DTYPE(train_labels_enc[:train_size])
x_val = FLOAT_DTYPE(train_data[train_size:])
y_val = INT_DTYPE(test_labels_enc[train_size:])

In [6]:
x_train = x_train.reshape(-1, 32, 32, 3)
x_val = x_val.reshape(-1, 32, 32, 3)

In [7]:
model = CNN()

model.add(Conv2D(3, 64))
model.add(MaxPoolLayer(2))
model.add(Dropout(0.5))
model.add(Relu())

model.add(Conv2D(3, 128))
model.add(MaxPoolLayer(2))
model.add(Dropout(0.5))
model.add(Relu())

model.add(Conv2D(3, 128))
model.add(MaxPoolLayer(2))
model.add(Dropout(0.5))
model.add(Softmax())

In [8]:
model.compile(x_train.shape)

In [9]:
model.train(x_train, y_train, EPOCHS)

Epoch 1 / 100 in progress =========================================
Epoch 1 Forward Propogation Started


  0%|          | 0/12 [00:00<?, ?it/s]

(40000, 32, 32, 3)
(40000, 30, 30, 3, 3, 3)
(3, 3, 3, 64)


  8%|▊         | 1/12 [03:03<33:38, 183.47s/it]

(40000, 30, 30, 64)


 33%|███▎      | 4/12 [04:41<09:23, 70.49s/it] 

(40000, 15, 15, 64)
(40000, 13, 13, 64, 3, 3)
(3, 3, 3, 128)


ValueError: Size of label 'k' for operand 1 (64) does not match previous terms (3).